In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

타이타닉과 유사한해상 사고가 발생했을 시 어떤 승객이 생존하며, 어떤 승객이 사망하는지를 예측하는 예측 모델 개발.

- Survival - 0이면 사망, 1이면 생존.
- Pclass - 티켓 등급. 1등석(1), 2등석(2), 3등석(3)
- Sex - 성별. 남자(male)와 여자(female).
- Age - 나이입니다. 틈틈히 빈 값이 존재하며, 소수점 값도 존재합니다.
- SibSp - 해당 승객과 같이 탑승한 형재/자매(siblings)와 배우자(spouses)의 총 인원 수.
- Parch - 해당 승객과 같이 탑승한 부모(parents)와 자식(children)의 총 인원 수.
- Ticket - 티켓 번호. 
- Fare - 운임 요금입니다. 소수점으로 구성되어 있습니다.
- Cabin - 객실 번호입니다. 많은 빈 값이 존재하며, 다양한 텍스트(문자열)로 구성되어 있습니다.
- Embarked - 선착장입니다. C는 셰르부르(Cherbourg)라는 프랑스 지역, Q는 퀸스타운(Queenstown)이라는 영국 지역, S는 사우스햄튼(Southampton).



In [ ]:
import pandas as pd

#  PassengerId를 index로 지정.
train = pd.read_csv("../input/taitanic/train.csv", index_col="PassengerId")

# train 변수에 할당된 데이터의 행렬 사이즈를 출력합니다.
# 출력은 (row, column) 으로 표시됩니다.
print(train.shape)

# head()로 train 데이터의 상위 5개를 띄웁니다.
train.head()

In [ ]:
test = pd.read_csv("../input/taitanic/test.csv", index_col="PassengerId")

print(test.shape)
test.head()

In [ ]:
# matplotlib & seaborn : 데이터 시각화 패키지
# matplotlib로 실행하는 모든 시각화를 자동으로 쥬피터 노트북에 띄웁니다.
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt


## Sex

In [ ]:
# countplot
sns.countplot(data=train, x="Sex", hue="Survived")

생존율 : Female > Male 

In [ ]:
# pivot_table
pd.pivot_table(train, index="Sex", values="Survived")

## Pclass

In [ ]:
# 객실등급
sns.countplot(data=train, x="Pclass", hue="Survived")

1등급: 생존자 수가 사망자 수보다 더 많다.

3등급: 사망자 수가 생존자수보다 압도적으로 높다.

In [ ]:
# pivot_table
pd.pivot_table(train, index="Pclass", values="Survived")

## Embarked

In [ ]:
sns.countplot(data=train, x="Embarked", hue="Survived")

S에서 탑승한 승객이 가장 많고 가장 많이 죽었다.

C에서 탑승한 승객은 생존율이 가장 높다.


In [ ]:
pd.pivot_table(train, index="Embarked", values="Survived")

## Age&Fare
*      상관관계 분석 lmplot

In [ ]:
sns.lmplot(data=train, x="Age", y="Fare", hue="Survived", fit_reg=False) #회귀선 false

오렌지 점(1)이 몰려 있는곳이 생존자가 많은 지점

동떨어져 있는 outlier들을 제거해보자.

In [ ]:
# 운임요금(Fare)이 500달러 미만인 데이터만 가져오자.
# 이를 low_fare라는 변수에 할당.
low_fare = train[train["Fare"] < 500]


# $500달러 이상을 지불한 세 명의 승객이 제거됌.
train.shape, low_fare.shape

In [ ]:
# data를 low_fare로 바꿔주자
sns.lmplot(data=low_fare, x="Age", y="Fare", hue="Survived", fit_reg=False)

## Preprocessing
    - 전처리
    - 데이터를 머신러닝 알고리즘이 이해할 수 있는 형태로 바꾸는 것
    - 모든 데이터는 숫자여야 함
    - 데이터에 빈 값이 없어야 함

## Encode Sex

Encoding

Male -> 0

Female -> 1

In [ ]:
# Sex_encode라는 컬럼을 따로 하나 만들어서 작업하자(사본)
train.loc[train["Sex"] == "male", "Sex_encode"] = 0
train.loc[train["Sex"] == "female", "Sex_encode"] = 1


print(train.shape)
train[["Sex", "Sex_encode"]].head()

In [ ]:
# Sex_encode라는 컬럼을 따로 하나 만들어서 작업하자(사본)
test.loc[test["Sex"] == "male", "Sex_encode"] = 0
test.loc[test["Sex"] == "female", "Sex_encode"] = 1


print(test.shape)
test[["Sex", "Sex_encode"]].head()

## Fare(운임)의 Nan값 채워넣기

scikit-learn에는 NaN이 있어선 안됌.
test의 NaN값을 채워주자

In [ ]:
train[train["Fare"].isnull()]
#빈값 없음

In [ ]:
test[test['Fare'].isnull()]
# 빈 값을 요금의 평균값으로 채우는 것이 일반적(train['Fare'].mean())
# 빈 값이 하나뿐이니 0을 넣자

In [ ]:
# 사본을 만들어서 저장
train["Fare_fillin"] = train["Fare"]

print(train.shape)
train[["Fare", "Fare_fillin"]].head()

In [ ]:

test["Fare_fillin"] = test["Fare"]

print(test.shape)
test[["Fare", "Fare_fillin"]].head()

In [ ]:
test.loc[test['Fare'].isnull(), 'Fare_fillin']=0

test.loc[test['Fare'].isnull(), ['Fare','Fare_fillin']] # Fare가 NaN인 loc를 찾아서, Fare와 Fare_filin칼럼만 보여달라

In [ ]:
test.loc[test['Fare'].isnull()]

## Encode Embarked

- C==0
- S==1
- Q==2
        으로 인코딩을 해주면 될까? 아니, 그러면 C+S가 가능하다는 뜻.
     
### One Hot Encoding     
- C==[1,0,0]
- S==[0,1,0]
- Q==[0,0,1]


In [ ]:
train["Embarked_C"] = train["Embarked"] == "C"
train["Embarked_S"] = train["Embarked"] == "S"
train["Embarked_Q"] = train["Embarked"] == "Q"


print(train.shape)
train[["Embarked", "Embarked_C", "Embarked_S", "Embarked_Q"]].head()

In [ ]:
test["Embarked_C"] = test["Embarked"] == "C"
test["Embarked_S"] = test["Embarked"] == "S"
test["Embarked_Q"] = test["Embarked"] == "Q"


print(test.shape)
test[["Embarked", "Embarked_C", "Embarked_S", "Embarked_Q"]].head()

### One Hot Encoding

C == [True, False, False] 

S == [False, True, False] 

Q == [False, False, True] 

## Train - Decision Tree

* Label: 맞춰야 하는 정답 (Survived)
* Feature: Label을 맞추는데 도움이 되는 값들

-

* Label : Survived
* Feature: Pclass, Sex_encode, Fare_fillin, Embarked
    - X_train : train의 Feature
    - Y_train : train의 Label
    - X_test : test의 Feature
    - Y_test : 이제 구해야 하는 값
    

In [ ]:
feature_names = ["Pclass", "Sex_encode", "Fare_fillin",
                 "Embarked_C", "Embarked_S", "Embarked_Q"]
feature_names

In [ ]:
label_name = "Survived"
label_name

In [ ]:
X_train = train[feature_names]
X_train.head()

In [ ]:
X_test = test[feature_names]
X_test.head()

In [ ]:
y_train = train[label_name]
y_train.head()

# 살아남았으면 1, 아니면 0

### SCikit-Learn

Fit : Tran의 Feature와 Label이 필요하다.

Predict : Fit이 끝나면 Predict가능. Test의 Feature값이 필요하다.

In [ ]:
from sklearn.tree import DecisionTreeClassifier

# depth의 깊이를 5로 설정
model = DecisionTreeClassifier(max_depth=5)
model


In [ ]:
# Fit : 모델을 train 데이터로 학습시키기 
model.fit(X_train, y_train)

### Visualize - Graphviz 툴

In [ ]:
import graphviz
from sklearn.tree import export_graphviz

# class_names. 살았을 경우 시각화에서 어떻게 표현할 것인지(Survived)
# out_file. 결과를 저장할 파일명. 지금은 바로 보여줄 것이라 저장 안함
dot_tree = export_graphviz(model,
                           feature_names=feature_names,
                           class_names=["Perish", "Survived"],
                           out_file=None)

#graphviz의 Source기능을 통해 Decision Tree 시각화
graphviz.Source(dot_tree)

### Predict

model.predict로 예측.

In [ ]:
# Fit을 마친뒤 predict기능으로 생존 예측
prediction = model.predict(X_test)

prediction[0:10]

캐글 제출 형태로 저장

캐글이 제공하는 제출 포맷을 이용 (gender_submission.csv)

In [ ]:
submission = pd.read_csv("../input/gender-submission/gender_submission.csv", index_col='PassengerId')

submission.head()

In [ ]:
submission["Survived"] = prediction

In [ ]:
# Csv파일로 저장
submission.to_csv("./2020_03_04_decision-tree_0.78947.csv")